In [1]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=440f2fe323a10eab9ef26de749431d7958b6db073d192932a0c07da9cb226966
  Stored in directory: c:\users\16513\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [1]:
import pandas as pd
import numpy as np

# keyword extraction and clean up
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

#Text Blob for sentiment analysis
from textblob import TextBlob

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import linear_kernel

import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect

ModuleNotFoundError: No module named 'rake_nltk'

In [6]:
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"

In [7]:
hotel_df = pd.read_csv(url)

In [8]:
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512470 entries, 0 to 512469
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Unnamed: 0                                  512470 non-null  int64  
 1   Hotel_Address                               512470 non-null  object 
 2   Additional_Number_of_Scoring                512470 non-null  int64  
 3   Review_Date                                 512470 non-null  object 
 4   Average_Score                               512470 non-null  float64
 5   Hotel_Name                                  512470 non-null  object 
 6   Reviewer_Nationality                        512470 non-null  object 
 7   Negative_Review                             512470 non-null  object 
 8   Review_Total_Negative_Word_Counts           512470 non-null  int64  
 9   Total_Number_of_Reviews                     512470 non-null  int64  
 

In [10]:
# combine reviews into one field
hotel_df["All_Reviews"] = hotel_df['Positive_Review'].astype(str) +"-"+ hotel_df["Negative_Review"]
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive,No real complaints the hotel was great great ...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive,Location was good and staff were ok It is cut...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative,Great location in nice surroundings the bar a...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive,Amazing location and building Romantic settin...


In [11]:
hotel_unique_df = hotel_df.drop_duplicates('Hotel_Name')
hotel_unique_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
405,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,18,2,9.6,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...
971,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,17,1,9.2,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",1 days,51.513734,-0.108751,UK,Positive,They have followed through special requests C...
2008,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,6,6,8.3,"[' Family with young children ', ' Triple Room...",1 days,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...
3778,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,231,1,9.2,"[' Leisure trip ', ' Group ', ' Comfort Double...",35 days,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...


In [12]:
hotel_unique_df.Hotel_Name.value_counts()

Hotel Arena                      1
Hotel 55                         1
Steigenberger Hotel Herrenhof    1
Hotel J ger                      1
Senator Hotel Vienna             1
                                ..
The Cleveland                    1
Grange Buckingham Hotel          1
Hilton London Olympia            1
H tel De Castiglione             1
Atlantis Hotel Vienna            1
Name: Hotel_Name, Length: 1475, dtype: int64

In [13]:
hotel_unique_df = hotel_unique_df.reset_index(drop=True)
hotel_unique_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,18,2,9.6,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...
2,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,17,1,9.2,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",1 days,51.513734,-0.108751,UK,Positive,They have followed through special requests C...
3,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,6,6,8.3,"[' Family with young children ', ' Triple Room...",1 days,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...
4,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,231,1,9.2,"[' Leisure trip ', ' Group ', ' Comfort Double...",35 days,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...


In [14]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from combined reviews
hotel_unique_pp_df = hotel_unique_df.copy()
key_words = []
name_key_words = []

r = Rake()

for index, row in hotel_unique_pp_df.iterrows():
    r.extract_keywords_from_text(row['All_Reviews'])
    key_words_dict_scores = r.get_word_degrees()
    keywords = list(key_words_dict_scores.keys())
    
    key_words.append(keywords)

    r.extract_keywords_from_text(row['Hotel_Name'])
    key_words_dict_scores = r.get_word_degrees()
    name_keywords = list(key_words_dict_scores.keys())
    
    name_key_words.append(name_keywords)
    
hotel_unique_pp_df["key_words"] = key_words
hotel_unique_pp_df["name_key_words"] = name_key_words

hotel_unique_pp_df.head()

NameError: name 'Rake' is not defined

In [15]:
hotel_unique_pp_clean_df = hotel_unique_pp_df.copy()

In [16]:
# clean up keywords: drop symbols and numbers, as well as find root words

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


hotel_unique_pp_clean_df['clean_keywords']=hotel_unique_pp_clean_df['key_words'].map(lambda s:preprocess(s)) 
hotel_unique_pp_clean_df.head()

NameError: name 'WordNetLemmatizer' is not defined

In [17]:
hotel_sub_df = hotel_unique_pp_clean_df.copy()

In [18]:
# run text blob in a loop

polarities = []
subjectivities = []

for index, row in hotel_sub_df.iterrows():
    sentence = row['All_Reviews']
    analysis = TextBlob(sentence).sentiment
    polarity = analysis[0]
    subjectivity = analysis[1]

    # populate lists
    polarities.append(polarity)
    subjectivities.append(subjectivity)


hotel_sub_df['sentiment_polarity'] = polarities
hotel_sub_df['sentiment_subjectivity']= subjectivities

hotel_sub_df.head()

NameError: name 'TextBlob' is not defined

In [19]:
hotel_sub_df.Hotel_Name.value_counts()

Hotel Arena                      1
Hotel 55                         1
Steigenberger Hotel Herrenhof    1
Hotel J ger                      1
Senator Hotel Vienna             1
                                ..
The Cleveland                    1
Grange Buckingham Hotel          1
Hilton London Olympia            1
H tel De Castiglione             1
Atlantis Hotel Vienna            1
Name: Hotel_Name, Length: 1475, dtype: int64

In [20]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [21]:
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

KeyError: 'clean_keywords'

In [22]:
tfidf_matrix

NameError: name 'tfidf_matrix' is not defined

In [23]:
#calculate cosine matrix
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

NameError: name 'tfidf_matrix' is not defined

In [24]:
indices = pd.Series(hotel_sub_df['Hotel_Name'])

In [25]:
#get recommendations based on tf-idf and cosine_similarities (hotel name)

def recommend_name(Hotel_Name, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
    recommended_hotel_score = []
         
    idx = indices[indices == Hotel_Name].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
        recommended_hotel_score.append(score_series[i])
        
    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
            'Score': recommended_hotel_score
                       }
    
    rec_df = pd.DataFrame(data)
    
    return rec_df

NameError: name 'cosine_similarities' is not defined

In [26]:
rec_filter_df = recommend_name('Copthorne Tara Hotel London Kensington')
rec_filter_df = rec_filter_df[(rec_filter_df['Rating'] > 8)]
rec_filter_df.head()

NameError: name 'recommend_name' is not defined

In [27]:
# prepping for Flask

In [28]:
#code that stays consistent
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"
hotel_df = pd.read_csv(url)

tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

KeyError: 'clean_keywords'

In [ ]:
np.save("hotel_cos_data.npy", cosine_similarities)

In [29]:
hotel_arr = np.load("hotel_cos_data.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'hotel_cos_data.npy'

In [30]:
print(hotel_arr[0:5])

NameError: name 'hotel_arr' is not defined

In [31]:
#uploaded .npy arr file to S3 bucket
hotel_arr_url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/hotel_cos_data.npy"

In [32]:
import s3fs 

ModuleNotFoundError: No module named 's3fs'

In [33]:
hotel_arr_s3= np.load(s3.open("ralston-hotel-data-bucket/hotel_cos_data.npy"))

NameError: name 's3' is not defined

In [34]:
print(hotel_arr_s3[0:5])

NameError: name 'hotel_arr_s3' is not defined

In [35]:
#example variable
Hotel_Name = "Hotel Arena"


#variable reliant code
indices = pd.Series(hotel_sub_df['Hotel_Name'])

recommended_hotel = []
recommended_hotel_address = []
recommended_hotel_rating = []
recommended_hotel_score = []

idx = indices[indices == Hotel_Name].index[0]

score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

top_10_indices = list(score_series.iloc[1:11].index)

for i in top_10_indices:
    recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
    recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
    recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
    recommended_hotel_score.append(score_series[i])
    

data = {'Hotel': recommended_hotel,
        'Address': recommended_hotel_address,
        'Rating': recommended_hotel_rating,
        'Score': recommended_hotel_score
        }

rec_df = pd.DataFrame(data)
rec_df = rec_df[(rec_df['Rating'] > 8)]

rec_df


NameError: name 'cosine_similarities' is not defined